<a href="https://colab.research.google.com/github/DataScienceHandsOn21/DataScienceHandsOn21/blob/main/Pyspark_Regression_EcommerceData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 31 kB/s 
     |████████████████████████████████| 198 kB 52.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=2757882e1439e5665735a6ffe0f5aaaaae6204579bdef81a4fd12f075177552e
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [19]:
path = '/content/Ecommerce_Customers.csv'

In [20]:
df = spark.read.csv(path=path,
                    sep=',',encoding='UTF-8',comment=None,
                    header=True,inferSchema=True)

In [23]:
df.show(8)

+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|               Email|             Address|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|       34.49726773|12.65565115|    39.57766802|         4.082620633|         587.951054|
|   hduke@hotmail.com|4547 Archer Commo...|       31.92627203|11.10946073|    37.26895887|         2.664034182|        392.2049334|
|    pallen@yahoo.com|24645 Valerie Uni...|       33.00091476|11.33027806|    37.11059744|         4.104543202|        487.5475049|
|riverarebecca@gma...|1414 David Throug...|       34.30555663|13.71751367|    36.72128268|         3.120178783|         581.852344|
|mstephens@davidso...|14023 Rodriguez P...|       33.33067252|12.79518855|  

In [26]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [27]:
#Data Columns
df.columns

['Email',
 'Address',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [29]:
#Total Number of Records/Rows
df.count()

500

In [32]:
#Statistical Summary
df.describe().show()

+-------+-----------------+--------------------+------------------+------------------+------------------+--------------------+-------------------+
|summary|            Email|             Address|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+-------+-----------------+--------------------+------------------+------------------+------------------+--------------------+-------------------+
|  count|              500|                 500|               500|               500|               500|                 500|                500|
|   mean|             null|                null|    33.05319351824|12.052487936928012|37.060445421080004|  3.5334615559300007|  499.3140382608002|
| stddev|             null|                null|0.9925631111602911|0.9942156084624618|1.0104889068105993|  0.9992775024367542|  79.31478155115914|
|    min|aaron04@yahoo.com|0001 Mack MillNor...|       29.53242897|       8.508152176|       33.91384725|          0.2

In [33]:
#Creating Model and importing the utilities
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [34]:
#Creating Feature Assembler
independent_features =VectorAssembler(inputCols=["Avg Session Length", "Time on App", "Time on Website", "Length of Membership"], outputCol="Independent Features")

In [35]:
output = independent_features.transform(df)

In [36]:
output.show()

+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+--------------------+
|               Email|             Address|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|Independent Features|
+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|       34.49726773|12.65565115|    39.57766802|         4.082620633|         587.951054|[34.49726773,12.6...|
|   hduke@hotmail.com|4547 Archer Commo...|       31.92627203|11.10946073|    37.26895887|         2.664034182|        392.2049334|[31.92627203,11.1...|
|    pallen@yahoo.com|24645 Valerie Uni...|       33.00091476|11.33027806|    37.11059744|         4.104543202|        487.5475049|[33.00091476,11.3...|
|riverarebecca@gma...|1414 David Throug...|       34.30555663|13.71751367|    36.7

In [37]:
#Showing Independent Feature
output.select("Independent Features").show()

+--------------------+
|Independent Features|
+--------------------+
|[34.49726773,12.6...|
|[31.92627203,11.1...|
|[33.00091476,11.3...|
|[34.30555663,13.7...|
|[33.33067252,12.7...|
|[33.87103788,12.0...|
|[32.0215955,11.36...|
|[32.73914294,12.3...|
|[33.9877729,13.38...|
|[31.93654862,11.8...|
|[33.99257277,13.3...|
|[33.87936082,11.5...|
|[29.53242897,10.9...|
|[33.19033404,12.9...|
|[32.38797585,13.1...|
|[30.73772037,12.6...|
|[32.1253869,11.73...|
|[32.33889932,12.0...|
|[32.18781205,14.7...|
|[32.61785606,13.9...|
+--------------------+
only showing top 20 rows



In [39]:
final_df = output.select("Independent Features", "Yearly Amount Spent")

In [40]:
final_df.show()

+--------------------+-------------------+
|Independent Features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.49726773,12.6...|         587.951054|
|[31.92627203,11.1...|        392.2049334|
|[33.00091476,11.3...|        487.5475049|
|[34.30555663,13.7...|         581.852344|
|[33.33067252,12.7...|         599.406092|
|[33.87103788,12.0...|        637.1024479|
|[32.0215955,11.36...|        521.5721748|
|[32.73914294,12.3...|        549.9041461|
|[33.9877729,13.38...|         570.200409|
|[31.93654862,11.8...|        427.1993849|
|[33.99257277,13.3...|        492.6060127|
|[33.87936082,11.5...|        522.3374046|
|[29.53242897,10.9...|        408.6403511|
|[33.19033404,12.9...|        573.4158673|
|[32.38797585,13.1...|        470.4527333|
|[30.73772037,12.6...|        461.7807422|
|[32.1253869,11.73...|        457.8476959|
|[32.33889932,12.0...|        407.7045475|
|[32.18781205,14.7...|        452.3156755|
|[32.61785606,13.9...|        605.0610388|
+----------

In [43]:
#Applying Linear Regression Algorithm
#Train and Test Split
train_data,test_data=final_df.randomSplit([0.75,0.25])

In [45]:
#Importing Linear Regression from pysparkML
from pyspark.ml.regression import LinearRegression

In [46]:
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Yearly Amount Spent')
regressor=regressor.fit(train_data)

In [50]:
#Cofficient of Regression
regressor.coefficients


DenseVector([25.6661, 38.7767, 0.7305, 61.7201])

In [51]:
#Regression Intercept
regressor.intercept

-1061.1028062102766

In [55]:
#Prediction
pred_=regressor.evaluate(test_data)
pred_.predictions.show(25)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+-------------------+------------------+
|Independent Features|Yearly Amount Spent|        prediction|
+--------------------+-------------------+------------------+
|[29.53242897,10.9...|        408.6403511|399.00316586502413|
|[30.4925367,11.56...|        282.4712457| 287.6202120899907|
|[30.57436368,11.3...|        442.0644138|442.58470141000953|
|[30.87948434,13.2...|           490.2066| 494.6824670754693|
|[31.04722214,11.1...|        392.4973992|388.94468831341464|
|[31.06132516,12.3...|        487.5554581| 494.1215746907583|
|[31.26064687,13.2...|        421.3266313| 422.6193311801205|
|[31.26810421,12.1...|        423.4705332|427.39585386412887|
|[31.42522688,13.2...|        530.7667187| 535.5321882848964|
|[31.5171218,10.74...|        275.9184207|  281.005422100038|
|[31.5261979,12.04...|        409.0945262| 419.0189907168649|
|[31.57020083,13.3...|        545.9454921| 564.1604943042248|
|[31.62536013,13.1...|        376.3369008| 381.2866654843565|
|[31.654

In [ ]:
---End of Analysis---